In [2]:
"""
KPI-Report MVP (analyst ▸ writer), Autogen 0.2.x
"""

from __future__ import annotations
import os
from autogen import ConversableAgent, LLMConfig
from autogen.agentchat import initiate_group_chat
from autogen.agentchat.group.patterns import AutoPattern

# ──────────────────────────────────────────────────────────────────────
# 1. LLM config  (uses your METIS wrapper)
# ──────────────────────────────────────────────────────────────────────
OPENAI_API_KEY = "tpsg-AYCymq4XvkEmdyoL1Na0cujGfY0aeqys"


MODEL_NAME = "gpt-4o-mini"

config_list = [
    {
        "model": MODEL_NAME,
        "api_key": "tpsg-AYCymq4XvkEmdyoL1Na0cujGfY0aeqy",
        "base_url": "https://api.metisai.ir/api/v1/wrapper/openai_chat_completion"
    }
]

llm_config = LLMConfig(config_list=config_list, temperature=1)

# ──────────────────────────────────────────────────────────────────────
# 2. Role prompts
# ──────────────────────────────────────────────────────────────────────
ANALYST_SYS_MSG = """
You are the ANALYST.

• Input: ≤ 5 JSON blobs with keys "name" and "data".
• Parse mentally (no code).
• Typical KPIs: impressions, conversions, clicks, revenue.
• Derive these if possible:
    conversion_rate = conversions / impressions × 100
    ctr             = clicks / impressions × 100
• Compare each KPI to any <target name="…">value</target>.
• Output:

    KPI_NAME
        current = …
        target  = … (or N/A)
        delta   = +X or −Y
        comment = …

• End with **ANALYSIS_DONE**.
"""

WRITER_SYS_MSG = """
You are the WRITER.

• You receive the analyst block (ends **ANALYSIS_DONE**).
• Produce Markdown:

## Executive Summary
<~2 sentences>

## KPI Details
- KPI A – …

## Recommendations
- 2-3 bullets if any KPI below target, else: “All KPIs meet or exceed target.”

Finish with **REPORT_DONE** (Markdown only).
"""

# ──────────────────────────────────────────────────────────────────────
# 3. Create agents
# ──────────────────────────────────────────────────────────────────────
with llm_config:                    # every agent inside inherits config
    analyst = ConversableAgent(
        name="analyst",
        system_message=ANALYST_SYS_MSG,
    )
    writer = ConversableAgent(
        name="writer",
        system_message=WRITER_SYS_MSG,
    )

user = ConversableAgent(            # human entry-point
    name="user",
    system_message="You provide KPI JSON and targets, then wait for the report.",
    llm_config=False,
    human_input_mode="TERMINATE",
)

# ──────────────────────────────────────────────────────────────────────
# 4. Conversation pattern (start with the analyst)
# ──────────────────────────────────────────────────────────────────────
pattern = AutoPattern(
    initial_agent=analyst,                # <-- key fix
    agents=[analyst, writer],
    user_agent=user,
    group_manager_args={"llm_config": llm_config},
)

# ──────────────────────────────────────────────────────────────────────
# 5. Helper: JSON list + targets  → Markdown report
# ──────────────────────────────────────────────────────────────────────
def generate_report(json_blobs: list[str], targets_xml: str = "") -> str:
    payload = "\n\n".join(f"```json\n{blob}\n```" for blob in json_blobs)
    prompt  = f"Here are the KPI blobs:\n{payload}\n\n{targets_xml}"

    chat, *_ = initiate_group_chat(pattern=pattern, messages=prompt)
    return chat[-1]["content"]            # writer’s Markdown (REPORT_DONE)

# ──────────────────────────────────────────────────────────────────────
# 6. Demo run
# ──────────────────────────────────────────────────────────────────────
if __name__ == "__main__":
    JSON_IMPRESSIONS = """
    {
      "name": "impressions",
      "data": [
        {"date": "2025-08-01", "value": 60000},
        {"date": "2025-08-08", "value": 51000},
        {"date": "2025-08-15", "value": 75000},
        {"date": "2025-08-22", "value": 68000},
        {"date": "2025-08-29", "value": 85000}
      ]
    }
    """

    JSON_CONVERSIONS = """
    {
      "name": "conversions",
      "data": [
        {"value": 1635}
      ]
    }
    """

    TARGETS = """
    <target name="impressions">65000</target>
    <target name="conversions">1500</target>
    """

    report_md = generate_report(
        json_blobs=[JSON_IMPRESSIONS, JSON_CONVERSIONS],
        targets_xml=TARGETS,
    )

    print(report_md)        # ends with **REPORT_DONE**


user (to chat_manager):

Here are the KPI blobs:
```json

    {
      "name": "impressions",
      "data": [
        {"date": "2025-08-01", "value": 60000},
        {"date": "2025-08-08", "value": 51000},
        {"date": "2025-08-15", "value": 75000},
        {"date": "2025-08-22", "value": 68000},
        {"date": "2025-08-29", "value": 85000}
      ]
    }
    
```

```json

    {
      "name": "conversions",
      "data": [
        {"value": 1635}
      ]
    }
    
```


    <target name="impressions">65000</target>
    <target name="conversions">1500</target>
    

--------------------------------------------------------------------------------

Next speaker: analyst


>>>>>>>> USING AUTO REPLY...
analyst (to chat_manager):

**KPI: Impressions**
- current = 65000 (calculated as average of weekly data: (60000 + 51000 + 75000 + 68000 + 85000) / 5)
- target = 65000
- delta = +0
- comment = The current impressions meet the target.

---

**KPI: Conversions**
- current = 1635
- target 

KeyboardInterrupt: 